In [1]:
import nglview as nv
import pandas as pd
import numpy as np
import MDAnalysis as mda
import mdtraj as md
import os
import scipy
import csv

import matplotlib.pyplot as plt

import rpy2.robjects as robjects
from IPython.display import Image

%load_ext rpy2.ipython

import warnings
warnings.filterwarnings ('error', category=DeprecationWarning)

/home/george/anaconda3/lib/python3.9/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version. The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


In [3]:
pdb = "step5_input.pdb"

psf = "step5_input.psf" 

dcd = "step7_production.dcd"

#u = mda.Universe (psf , dcd)

traj =  md.load(dcd, top = psf)

Bellow we will see two 'Cells'. One will correspond to TM1 through both monomers within the EnvZ dimer; the other TM2. Please remove the '#' marks from the TM which you wish to run. 

Rerun the notebook and alternate the '#' positions when you would like to move between the TMs. (: 



In [3]:

# FOR TM1 PLEASE RUN THIS CELL


proa = u.select_atoms ("protein and segid PROA and resid 11 : 41")

prob = u.select_atoms ("protein and segid PROB and resid 11 : 41")

output_csv_file = 'TM1_Replicate_1_Inter_Residue_Distances.csv'


proaTraj = traj.topology.select('protein and segname PROA and resid 11 to 41')

probTraj = traj.topology.select('protein and segname PROB and resid 11 to 41')



EnvZ_Monomer_A = md.rmsd(proaTraj , proaTraj, 0)
EnvZ_Monomer_B = md.rmsd(probTraj,  probTraj, 0)

fig1= plt.plot(Global_RMSD, color='orange', label='Global')
plt.plot(EnvZ_Monomer_A, color='brown', label='Monomer A')
plt.plot(EnvZ_Monomer_B, color='red', label='Monomer B')

plt.xlabel('Time (ns)')
plt.ylabel('RMSD (nm)')
plt.title('RMSD of EnvZ. Monomers and Global')

plt.legend(loc='lower right')

AttributeError: 'numpy.ndarray' object has no attribute 'xyz'

In [1]:
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt

# Load the trajectory and topology
traj = md.load(dcd, top=pdb)

# Get the atom indices for the selections
proa_indices = traj.topology.select('protein and segname PROA and resid 11 to 41')
prob_indices = traj.topology.select('protein and segname PROB and resid 11 to 41')

# Calculate RMSD for the whole system
Global_RMSD = md.rmsd(traj, traj, 0)

# Calculate RMSD for Monomer A
EnvZ_Monomer_A = md.rmsd(traj, traj, 0, atom_indices=proa_indices)

# Calculate RMSD for Monomer B
EnvZ_Monomer_B = md.rmsd(traj, traj, 0, atom_indices=prob_indices)

# Plot RMSD
fig1 = plt.figure(figsize=(10, 6))
plt.plot(Global_RMSD, color='orange', label='Global')
plt.plot(EnvZ_Monomer_A, color='brown', label='Monomer A')
plt.plot(EnvZ_Monomer_B, color='red', label='Monomer B')

plt.xlabel('Time (ns)')
plt.ylabel('RMSD (nm)')
plt.title('RMSD of EnvZ Monomers and Global')

plt.legend(loc='lower right')
plt.show()


NameError: name 'dcd' is not defined

In [ ]:

# FOR TM2 PLEASE RUN THIS CELL

#proa = u.select_atoms ("protein and segid PROA and resid 156 : 183")

#prob = u.select_atoms ("protein and segid PROB and resid 156 : 183")

#output_csv_file = 'TM2_Replicate_1_Inter_Residue_Distances.csv'

In [ ]:
frames = len(u.trajectory)
residues = len(proa)
distances = np.zeros((frames, residues))
time_step_ps = u.trajectory.dt

In [ ]:
for idx, ts in enumerate(u.trajectory):
    for i in range(residues):
        distance = np.linalg.norm(proa.positions[i] - prob.positions[i])
        distances[idx, i] = distance

In [ ]:
with open(output_csv_file, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    
    
    header = ['Time (ps)'] + [f'Residue {i + 11}' for i in range(residues)]
    csvwriter.writerow(header)
    
    for idx in range(frames):
        row = [idx * time_step_ps] + list(distances[idx])
        csvwriter.writerow(row)

In [ ]:
r_code = """
library(ggplot2)
library(reshape2)
library(repr)

data <- read.csv('TM1_Replicate_1_Inter_Residue_Distances.csv')

residues <- 11:41
data <- data[, c('Time..ps.', paste0('Residue.', residues))]

melted_data <- melt(data, id.vars = 'Time..ps.')

box_plot <- ggplot(melted_data, aes(x = variable, y = value)) +
  geom_boxplot() +
  labs(x = 'Residue', y = 'Inter-Residue Distance (Å)',
       title = 'Inter residue distance of the TM1 domain within an EnvZ Homodimer') +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

options(repr.plot.width=10, repr.plot.height=6)

plot_path <- 'box_plot.png'
ggsave(plot_path, plot=box_plot, width=10, height=6)
plot_path
"""

output = robjects.r(r_code)
plot_path = output[0]
Image(filename=plot_path)


In [ ]:
r_code = """
library(ggplot2)
library(reshape2)
library(repr)

data <- read.csv('TM1_Replicate_1_Inter_Residue_Distances.csv')

residues <- 11:41
data <- data[, c('Time..ps.', paste0('Residue.', residues))]

melted_data <- melt(data, id.vars = 'Time..ps.')

box_plot <- ggplot(melted_data, aes(x = variable, y = value, fill = variable)) +
  geom_boxplot() +
  labs(x = 'Residue', y = 'Inter-Residue Distance (Å)',
       title = 'Inter residue distance of the TM1 domain within an EnvZ Homodimer') +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  guides(fill = FALSE)

options(repr.plot.width=10, repr.plot.height=6)

plot_path <- 'box_plot.png'
ggsave(plot_path, plot=box_plot, width=10, height=6)
plot_path
"""

output = robjects.r(r_code)
plot_path = output[0]
Image(filename=plot_path)


In [ ]:
r_code = """
library(ggplot2)
library(reshape2)
library(repr)

data <- read.csv('TM2_Replicate_1_Inter_Residue_Distances.csv')

residues <- 156:183
data <- data[, c('Time..ps.', paste0('Residue.', residues))]

melted_data <- melt(data, id.vars = 'Time..ps.')

box_plot <- ggplot(melted_data, aes(x = variable, y = value)) +
  geom_boxplot() +
  labs(x = 'Residue', y = 'Inter-Residue Distance (Å)',
       title = 'Box Plot of Inter-Residue Distances between PROA and PROB Residues') +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

options(repr.plot.width=10, repr.plot.height=6)

plot_path <- 'box_plot.png'
ggsave(plot_path, plot=box_plot, width=10, height=6)
plot_path
"""

output = robjects.r(r_code)
plot_path = output[0]
Image(filename=plot_path)


In [ ]:
r_code = """
library(ggplot2)
library(reshape2)
library(repr)

data <- read.csv('TM2_Replicate_1_Inter_Residue_Distances.csv')

residues <- 156:183
data <- data[, c('Time..ps.', paste0('Residue.', residues))]

melted_data <- melt(data, id.vars = 'Time..ps.')

box_plot <- ggplot(melted_data, aes(x = variable, y = value, fill = variable)) +
  geom_boxplot() +
  labs(x = 'Residue', y = 'Inter-Residue Distance (Å)',
       title = 'Inter residue distance of the TM2 domain within an EnvZ Homodimer') +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  guides(fill = FALSE)

options(repr.plot.width=10, repr.plot.height=6)

plot_path <- 'box_plot.png'
ggsave(plot_path, plot=box_plot, width=10, height=6)
plot_path
"""

output = robjects.r(r_code)
plot_path = output[0]
Image(filename=plot_path)
